# Conformal Object Detection: first steps

This tutorial should get you started doing **Conformal Object Detection (COD)** with the [`cods`](https://github.com/leoandeol/cods) library.

For more information on the methods implemented in CODS, see the papers: 
- [Andéol et al. 2023: Confident Object Detection via Conformal Prediction and Conformal Risk Control](https://proceedings.mlr.press/v204/andeol23a.html)
- [Angelopoulos et al. 2022: Conformal Risk Control](https://arxiv.org/abs/2208.02814)
- [Li et al. 2022: Towards PAC Multi-Object Detection and Tracking](https://arxiv.org/abs/2204.07482)
- [Bates et al. 2021: Risk Controlling Prediction Sets](https://dl.acm.org/doi/abs/10.1145/3478535)


### Get started
1. Download the MS-COCO dataset: 
    - https://cocodataset.org/
2. Download DETR: automatically via Pytorch hub: https://pytorch.org/hub/
    - source: https://github.com/facebookresearch/detr

### Contents
What we will be doing:
1. Setup inference [⤵](#Setup-inferences)
    - load predictor (DETR) pretrained on COCO
    - Split the validation into: calibration & validation dataset
2. Run inferences on these datasets [⤵](#Setup-inferences)
    - Save predictions to disk: faster than re-predict for every test
3. Test Conformal Prediction !

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from cods.od.data import MSCOCODataset
from cods.od.models import DETRModel
import logging
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = (
    "0"  # chose the GPU. If only one, then "0"
)

logging.getLogger().setLevel(logging.INFO)

## 2. Setup inferences [🔝](#conformal-object-detection-first-steps)

In [3]:
# set [COCO_PATH] to the directory to your local copy of the COCO dataset
COCO_PATH = "/datasets/shared_datasets/coco/"

data = MSCOCODataset(root=COCO_PATH, split="val")

In [4]:
calibration_ratio = (
    0.5  # set 0.5 to use 50% for calibration and 50% for testing
)

use_smaller_subset = True  # TODO: Temp

if use_smaller_subset:
    data_cal, data_val = data.random_split(
        calibration_ratio, shuffled=False, n_calib_test=800
    )
else:
    data_cal, data_val = data.random_split(calibration_ratio, shuffled=False)

# model and weights are downloaded from https://github.com/facebookresearch/detr
detr = DETRModel(model_name="detr_resnet50", pretrained=True)

print(f"{len(data) = }")
print(f"{len(data_cal) = }")
print(f"{len(data_val) = }")

Using cache found in /home/leo.andeol/.cache/torch/hub/facebookresearch_detr_main
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


len(data) = 5000
len(data_cal) = 400
len(data_val) = 400


Run inferences:
- the first time, run inferences and save them disk
- if predictions are saved on disk, load them

In [5]:
preds_cal = detr.build_predictions(
    data_cal,
    dataset_name="mscoco",
    split_name="cal",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=False,
)
preds_val = detr.build_predictions(
    data_val,
    dataset_name="mscoco",
    split_name="test",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=False,
)

Loading predictions from ./saved_predictions/detr_resnet50/mscoco/cal/predictions_object_detection.pkl
Predictions already exist, loading them...
Loading predictions from ./saved_predictions/detr_resnet50/mscoco/test/predictions_object_detection.pkl
Predictions already exist, loading them...


# New Tests

In [6]:
from cods.od.cp import ODConformalizer

conf = ODConformalizer(
    backend="auto",
    guarantee_level="object",
    matching="assymetric_hausdorff",
    multiple_testing_correction=None,
    confidence_method="nb_boxes",
    localization_method="boxwise",
    localization_prediction_set="additive",
    classification_method="lac",
    optimizer="binary_search",
)

/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[WARNING:cp.py:cp:1217 - __init__ ] No multiple_testing_correction provided, assuming no correction is needed. The explicit list of alphas is expected for calibration.
[INFO:cp.py:cp:181 - __init__ ] Defaulting to CRC backend
[INFO:cp.py:cp:900 - __init__ ] Defaulting to CRC backend


In [7]:
# TODO(leo): we can replace this by anything, doesn't even need a guarantee (confidence)
parameters = conf.calibrate(
    preds_cal,
    alpha_confidence=0.01,
    alpha_localization=0.05,
    alpha_classification=0.05,
)

[INFO:cp.py:cp:1429 - calibrate ] Calibrating Confidence Conformalizer
  0%|          | 0/13 [00:00<?, ?it/s]

[0.08, 0.08] -> [0.0770263671875]. Corrected Risk = 0.00: 100%|██████████| 13/13 [02:00<00:00,  9.26s/it]
[0.08, 0.08] -> [0.0770263671875]. Corrected Risk = 0.01: 100%|██████████| 13/13 [01:59<00:00,  9.21s/it]
[INFO:cp.py:cp:1439 - calibrate ] Setting Confidence Threshold of Predictions
[INFO:cp.py:cp:1448 - calibrate ] Calibrated Confidence λ : 0.0770
	 and associated Confidence Threshold : 0.9229736328125
[WARNING:cp.py:cp:1457 - calibrate ] Overwriting previous matching
[INFO:cp.py:cp:1459 - calibrate ] Matching Predictions to True Boxes


Using assymetric hausdorff distance


400it [00:05, 69.36it/s]
[INFO:cp.py:cp:1471 - calibrate ] Calibrating Localization Conformalizer
[INFO:cp.py:cp:448 - calibrate ] Using overload confidence threshold: 0.9230
[248.29, 248.54] -> [248.4130859375]. Corrected Risk = 0.05: 100%|██████████| 13/13 [00:18<00:00,  1.40s/it]
[INFO:cp.py:cp:468 - calibrate ] Calibrated λ for localization: 248.53515625
[INFO:cp.py:cp:1481 - calibrate ] Calibrated Localization λ : 248.53515625
[INFO:cp.py:cp:1489 - calibrate ] Calibrating Classification Conformalizer
[INFO:cp.py:cp:1018 - calibrate ] Using overload confidence threshold: 0.9230
[WARNING:cp.py:cp:1023 - calibrate ] Currently considering that there is only one matching prediction to each true box for classification pruposes. To add later how to aggregate if multiple preidctions matched.
[1.00, 1.00] -> [0.9999996801489033]. Corrected Risk = 0.05: 100%|██████████| 40/40 [00:49<00:00,  1.23s/it]
[INFO:cp.py:cp:1042 - calibrate ] Calibrated λ for classification: 0.9999996802071109
[INFO

In [8]:
conformal_preds = conf.conformalize(preds_val, parameters=parameters)

[INFO:cp.py:cp:1541 - conformalize ] Conformalizing Predictions
[INFO:cp.py:cp:1544 - conformalize ] Using provided parameters for conformalization
[INFO:cp.py:cp:1550 - conformalize ] The parameters have been computed on another set of predictions.
[INFO:cp.py:cp:1562 - conformalize ] Conformalizing Confidence
[INFO:cp.py:cp:1578 - conformalize ] Conformalizing Localization
[INFO:cp.py:cp:513 - conformalize ] Using previous λ for localization
[INFO:cp.py:cp:528 - conformalize ] Conformalizing Localization with λ
[INFO:cp.py:cp:1588 - conformalize ] Conformalizing Classification


In [9]:
# TODO: Rewrite it so we only compute the confidence loss and not the max of three. Main loss of condiecne shoudl be just itself but in calibration use the proxy maximum loss with the others
results_val = conf.evaluate(
    preds_val,
    parameters=parameters,
    conformalized_predictions=conformal_preds,
    include_confidence_in_global=False,
)

[INFO:cp.py:cp:1627 - evaluate ] Evaluating Confidence Conformalizer
[INFO:cp.py:cp:1640 - evaluate ] Evaluating Localization Conformalizer
[INFO:cp.py:cp:1653 - evaluate ] Evaluating Classification Conformalizer
[WARNING:cp.py:cp:1074 - evaluate ] Evaluating classification conformalizer
100%|██████████| 400/400 [00:00<00:00, 691.42it/s]
[INFO:cp.py:cp:1682 - evaluate ] Evaluation Results:
[INFO:cp.py:cp:1684 - evaluate ] 	 Confidence:
[INFO:cp.py:cp:1685 - evaluate ] 		 Risk: 0.08
[INFO:cp.py:cp:1686 - evaluate ] 		 Mean Set Size: 70.07
[INFO:cp.py:cp:1690 - evaluate ] 	 Localization:
[INFO:cp.py:cp:1691 - evaluate ] 		 Risk: 0.05
[INFO:cp.py:cp:1692 - evaluate ] 		 Mean Set Size: 611.44
[INFO:cp.py:cp:1696 - evaluate ] 	 Classification:
[INFO:cp.py:cp:1697 - evaluate ] 		 Risk: 0.04
[INFO:cp.py:cp:1698 - evaluate ] 		 Mean Set Size: 38.02
[INFO:cp.py:cp:1702 - evaluate ] 	 Global:
[INFO:cp.py:cp:1703 - evaluate ] 		 Risk: 0.94


In [10]:
[INFO:cp.py:cp:1682 - evaluate ] Evaluation Results:
[INFO:cp.py:cp:1684 - evaluate ] 	 Confidence:
[INFO:cp.py:cp:1685 - evaluate ] 		 Risk: 0.08
[INFO:cp.py:cp:1686 - evaluate ] 		 Mean Set Size: 70.07
[INFO:cp.py:cp:1690 - evaluate ] 	 Localization:
[INFO:cp.py:cp:1691 - evaluate ] 		 Risk: 0.05
[INFO:cp.py:cp:1692 - evaluate ] 		 Mean Set Size: 422.46
[INFO:cp.py:cp:1696 - evaluate ] 	 Classification:
[INFO:cp.py:cp:1697 - evaluate ] 		 Risk: 0.04
[INFO:cp.py:cp:1698 - evaluate ] 		 Mean Set Size: 41.66
[INFO:cp.py:cp:1702 - evaluate ] 	 Global:
[INFO:cp.py:cp:1703 - evaluate ] 		 Risk: 0.92

SyntaxError: invalid syntax (1757850753.py, line 1)

# 3. Test Conformal Prediction

# Sebastien

In [ ]:
from cods.od.cp import ODRiskConformalizer

odc = ODRiskConformalizer(
    localization_method="pixelwise",
    # objectness_method="box_number",
    # classification_method="lac",
    # multiple_testing_correction="bonferroni",
    confidence_threshold=0.2,
)
odc.calibrate(preds_cal, alpha=0.1)
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.cp import ODRiskConformalizer

odc = ODRiskConformalizer(
    # localization_method="pixelwise",
    # objectness_method="box_number",
    classification_method="lac",
    # multiple_testing_correction="bonferroni",
    confidence_threshold=0.2,
)
odc.calibrate(preds_cal, alpha=0.1)
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.cp import ODRiskConformalizer

odc = ODRiskConformalizer(
    localization_method="pixelwise",
    # objectness_method="box_number",
    confidence_threshold=0.2,
    classification_method="lac",
    multiple_testing_correction="bonferroni",
)
odc.calibrate(preds_cal, alpha=0.1)
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.visualization import plot_preds

plot_preds(preds_val, 1, conf_boxes, conf_cls)

# General

In [ ]:
from cods.od.cp import ODConformalizer

odc = ODConformalizer(
    localization_method="min-hausdorff-additive",
    objectness_method="box_number",
    classification_method="lac",
    multiple_testing_correction="bonferroni",
)
cal_output = odc.calibrate(preds_cal, alpha=0.1)
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.cp import ODRiskConformalizer

odc = ODRiskConformalizer(
    localization_method="pixelwise",
    objectness_method="box_number",
    classification_method="lac",
    multiple_testing_correction="bonferroni",
)
odc.calibrate(preds_cal, alpha=0.1)
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.tr import ODToleranceRegion

odc = ODToleranceRegion(
    localization_loss="pixelwise",
    confidence_loss="box_number",
    classification_loss="lac",
    multiple_testing_correction="bonferroni",
    inequality="bernstein",
)
odc.calibrate(preds_cal, alpha=0.2, delta=0.1, bounds=[0, 1000])
conf_boxes, conf_cls = odc.conformalize(preds_val)
metrics = odc.evaluate(preds_val, conf_boxes, conf_cls)

In [ ]:
from cods.od.visualization import plot_preds

plot_preds(preds_val, 10, conf_boxes=conf_boxes, conf_cls=conf_cls)

In [ ]:
from cods.od.metrics import unroll_metrics

unroll_metrics(od_preds=preds_val, conf_boxes=conf_boxes, conf_cls=conf_cls)